# JS 如何通过事件循环机制（EventLoop）实现异步执行


### 调用栈和堆


执行上下文被创建之后会被压入调用栈，调用栈是跟踪代码执行流程的数据结构，当代码开始执行的时候会首先创建全局执行上下文并入栈，
执行到函数的时候会创建函数执行上下文并入栈，调用栈只执行栈顶的执行上下文代码，栈顶函数执行完就出栈，继续执行调用栈的栈顶函数。

在执行上下文里，基本类型的变量值存在环境记录里，引用类型和函数的值存在堆内存里，环境记录里存的是堆的内存地址。

JS 是同步编程语言，却能实现异步执行。主要是通过事件循环实现的。
